In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import argparse
import matplotlib.pyplot as plt
import numpy as np
import pylab
import pickle
import random
import types

from qiskit import Aer, BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.optimizers import SPSA, COBYLA, L_BFGS_B
from qiskit.aqua.components.variational_forms import RY, RYRZ
#from qiskit.aqua.operators import Z2Symmetries
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.core import Hamiltonian, TransformationType, QubitMappingType
from time import time

args = types.SimpleNamespace()
#args.algorithm='VQE' 
#args.basis_set='321++g'
#args.basis_set='431g'
#args.basis_set='321g'
#args.basis_set='ano'
#args.basis_set='augpc0'
#args.basis_set='ccpvdz' 
#args.basis_set='def2svp'
#args.basis_set='demon'
#args.basis_set='dz'
#args.basis_set='dzp'
#args.basis_set='minao'
#args.basis_set='pc0'
#args.basis_set='pcseg0'
#args.basis_set='sto6g'
#args.basis_set='stuttgart'
#args.basis_set='tzv'

args.max_parallel_threads=10 
args.molecule='H2' 
args.num_shots=1000
#args.outpath_mol='/home/ubuntu/mol2qpu/output/operators/molecule/'
#args.outpath_ferm='/home/ubuntu/mol2qpu/output/operators/fermionic/'
#args.outpath_qub='/home/ubuntu/mol2qpu/output/operators/qubit/'
#args.outpath_vqe='/home/ubuntu/mol2qpu/output/VQE_results/'
args.outpath_ee='/home/ubuntu/mol2qpu/output/EE_results/'
args.qubitmapping_type='bravyi_kitaev'
#args.qubitmapping_type='jordan_wigner'
#args.qubitmapping_type='parity'
#args.random_seed=750 
#args.two_qubit_reduce = True
#args.vqe_aer = True
args.vqe_depth=1 
args.vqe_entangler='linear' 
#args.vqe_entangler='full' 
#args.vqe_max_iter=2
#args.vqe_opt_params = False
#args.vqe_optimizer='SPSA' 
#args.vqe_sim = True
#args.vqe_var_form='RY' 

## load qmolecule
d_string = '0pt5'
#filename_mol = args.outpath_mol + args.molecule + '_' + args.basis_set + '_' + d_string + '_MOLE' + '.pkl'
#qmolecule = pickle.load(open(filename_mol,'rb'))

## load operators
#filename_ferm = args.outpath_ferm + args.molecule + '_' + args.basis_set + '_' + d_string + '_FERM' + '.pkl'
#filename_qub = args.outpath_qub + args.molecule + '_' + args.basis_set + '_' + d_string + '_QUBIT_bk' + '.pkl'
#ferOp = pickle.load(open(filename_ferm, 'rb'))
#qubitOp = pickle.load(open(filename_qub, 'rb'))
molecule = 'H .0 .0 -0.25; H .0 .0 0.25'
basis_sets = ['sto3g', '321g']
optimizers = [COBYLA, L_BFGS_B, SPSA]

converge_cnts = np.empty(shape=[len(basis_sets), len(optimizers)], dtype=object)
converge_vals = np.empty(shape=[len(basis_sets), len(optimizers)], dtype=object)
ref = np.empty(shape=len(basis_sets), dtype=object)

In [ ]:
if args.qubitmapping_type == 'bravyi_kitaev':
    maptype = 'bk'
elif args.qubitmapping_type == 'jordan_wigner':
    maptype = 'jw'
elif args.qubitmappint_type == 'parity':
    maptype = 'par'

In [2]:
for i in range(len(basis_sets)):
    print("i ======= ", i)

    driver = PySCFDriver(molecule, basis=basis_sets[i])
    qmolecule = driver.run()
    operator =  Hamiltonian(transformation=TransformationType.FULL,
                            qubit_mapping=QubitMappingType.PARITY,
                            two_qubit_reduction=False,
                            freeze_core=False,
                            orbital_reduction=None)
    qubit_op, aux_ops = operator.run(qmolecule)
    
    ## Exact Eigensolver Energy Calculation
    ee = ExactEigensolver(qubit_op)
    ee_result = ee.run()
    ref[i] = ee_result['energy']
    filename_ee = args.outpath_ee \
                      + 'H2/'  \
                      + basis_sets[i] + '_' \
                      + d_string + '.pkl' 
        
    filehandler_ee = open(filename_ee, 'wb')
    pickle.dump(ref[i], filehandler_ee)
    print('Reference value: {}'.format(ref[i]))
    
    num_particles = qmolecule.num_alpha + qmolecule.num_beta 
    print("H2, basis_set = ", basis_sets[i])
    print("num orbitals = ", qmolecule.num_orbitals)
    print("num_particles = ", num_particles)
    print("num qubits = ", qubit_op.num_qubits)
        
#     for j in range(len(optimizers)):
#         print("j ======= ", j)
#         aqua_globals.random_seed = 250
#         optimizer = optimizers[j]()
#         print('\rOptimizer: {}        '.format(type(optimizer).__name__), end='\n')
        
#          #var_form = RY(num_qubits, initial_state=init_state)
#         var_form = RYRZ(qubit_op.num_qubits, depth=args.vqe_depth, entanglement=args.vqe_entangler) 

#         counts = []
#         values = []
#         def store_intermediate_result(eval_count, parameters, mean, std):
#             counts.append(eval_count)
#             values.append(mean)
    
#         algo = VQE(qubit_op, var_form, optimizer, callback=store_intermediate_result)
#         backend = Aer.get_backend('qasm_simulator')
#         quantum_instance = QuantumInstance(circuit_caching=True, backend=backend,
#                                            backend_options={'max_parallel_threads': args.max_parallel_threads,                                                                             'max_parallel_experiments': 0, 
#                                                             'shots': args.num_shots})
#         algo_result = algo.run(quantum_instance)
#         #converge_cnts[i][j] = np.asarray(counts)
#         #converge_vals[i][j] = np.asarray(values)
        
#         ## save VQE results
#         filename_vqe = args.outpath_vqe \
#                      + 'H2/'  \
#                      + type(optimizer).__name__  + '/' \
#                      + 'VQE_' \
#                      + basis_sets[i] + '_' \
#                      + d_string + '_' \
#                      + 'RY_lin_bk_'  \
#                      + str(args.num_shots) + '.pkl'
        
#         filehandler_vqe = open(filename_vqe, 'wb')
#         pickle.dump(algo_result, filehandler_vqe)
        
#         filename_counts = args.outpath_vqe \
#                             + 'H2/'  \
#                             + type(optimizer).__name__  + '/' \
#                             + 'CNTS_' \
#                             + basis_sets[i] + '_' \
#                             + d_string + '_' \
#                             + 'RY_lin_bk_'  \
#                             + str(args.num_shots) + '.pkl'
        
#         filehandler_counts = open(filename_counts, 'wb')
#         pickle.dump(counts, filehandler_counts)
        
#         filename_vals = args.outpath_vqe \
#                             + 'H2/'  \
#                             + type(optimizer).__name__  + '/' \
#                             + 'VALS_' \
#                             + basis_sets[i] + '_' \
#                             + d_string + '_' \
#                             + 'RY_lin_bk_'  \
#                             + str(args.num_shots) + '.pkl'
        
#         filehandler_vals = open(filename_vals, 'wb')
#         pickle.dump(values, filehandler_vals)
        
#         print("saved file = ", filename_vqe)
#         print("saved file = ", filename_vals)
#         print("saved file = ", filename_counts)
    
    print('\rOptimization complete      ')

i =======  0
Reference value: -2.113514216310622
H2, basis_set =  sto3g
num orbitals =  2
num_particles =  2
num qubits =  4
Optimization complete      
i =======  1
Reference value: -2.1295678571917978
H2, basis_set =  321g
num orbitals =  4
num_particles =  2
num qubits =  8
Optimization complete      


In [7]:
basis_sets = ['321g']
driver = PySCFDriver(molecule, basis=basis_sets[0])
qmolecule = driver.run()
operator =  Hamiltonian(transformation=TransformationType.FULL,
                            qubit_mapping=QubitMappingType.PARITY,
                            two_qubit_reduction=False,
                            freeze_core=False,
                            orbital_reduction=None)
qubit_op, aux_ops = operator.run(qmolecule)
    
    ## Exact Eigensolver Energy Calculation
ee = ExactEigensolver(qubit_op)
ee_result = ee.run()
ref = ee_result['energy']
filename_ee = args.outpath_ee \
                      + 'H2/'  \
                      + basis_sets[0] + '_' \
                      + d_string + '.pkl' 
        
filehandler_ee = open(filename_ee, 'wb')
pickle.dump(ref, filehandler_ee)
print('Reference value: {}'.format(ref))

Reference value: -2.129567857191801
